# Mostra todas as classificações

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, Markdown
import time
import random

# Criando dicionário de épocas
epocas = {}
for i in range(63, 156):
    epoca = i - 63
    ano_inicio = 1933 + epoca
    ano_fim = ano_inicio + 1
    epocas[i] = f"{ano_inicio}-{str(ano_fim)[-2:]}"

def extrair_titulo_subtitulo(url_competicao):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url_competicao, headers=headers)
        response.raise_for_status()

        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, 'html.parser')

        subtitle = soup.select_one("#zz-enthdr > div.zz-enthdr-top > div.zz-enthdr-data > div.text")
        title = soup.select_one("#zz-enthdr > div.zz-enthdr-top > div.zz-enthdr-data > h1 > span")
        
        title = title.text.strip() if title else '<span style="color:red;">Título não encontrado</span>'
        subtitle = subtitle.text.strip() if subtitle else None
        
        return title, subtitle
    except:
        return '<span style="color:red;">Erro ao carregar a página</span>', None

def extrair_classificacao(url_competicao):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        time.sleep(random.uniform(1, 3))
        
        response = requests.get(url_competicao, headers=headers)
        response.raise_for_status()

        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, 'html.parser')
        
        table_element = None
        
        table_element = soup.find('table', class_='zztable stats zz-datatable')
        
        if not table_element:
            edition_table_div = soup.find('div', id='edition_table')
            if edition_table_div:
                table_element = edition_table_div.find('table')
        
        if not table_element:
            for table in soup.find_all('table'):
                headers = table.find_all('th')
                if headers and len(headers) >= 10:
                    header_text = [h.text.strip() for h in headers]
                    if 'P' in header_text and 'J' in header_text and 'V' in header_text:
                        table_element = table
                        break
        
        if not table_element:
            raise ValueError("Tabela de classificação não encontrada.")
        
        rows = table_element.find_all('tr')
        data = []
        for row in rows[1:]:
            cols = row.find_all('td')
            if len(cols) >= 10:
                row_data = [col.text.strip() for col in cols[:10]]
                data.append(row_data)
        
        df = pd.DataFrame(data, columns=['Posição', ' ', 'Equipa', 'Pontos', 'V', 'E', 'D', 'GM', 'GS', 'DG'])
        
        return df
    
    except requests.RequestException as e:
        print(f"Erro na requisição HTTP para {url_competicao}: {e}")
    except ValueError as e:
        print(f"Erro ao processar os dados de {url_competicao}: {e}")
    except Exception as e:
        print(f"Erro inesperado ao extrair classificação de {url_competicao}: {e}")
    
    return None

urls_competicoes = [
    "https://www.zerozero.pt/edicao/ii-1f-zona-d-grupo-8-34-35/93203",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-d-1f-grupo-2-35-36/93189",
    "https://www.zerozero.pt/edicao/ii-divisão-1fase-série-10-36-37/93375",
    "https://www.zerozero.pt/edicao/ii-divisão-série-10-1937-38/133915",
    "https://www.zerozero.pt/edicao/ii-divisao-serie-algarve-1938-39/133916",
    "https://www.zerozero.pt/edicao/ii-divisão-série-algarve-1939-40/134232",
    "https://www.zerozero.pt/edicao/ii-divisão-algarve-1940-41/132846",
    "https://www.zerozero.pt/edicao/ii-divisão-grupo-d-série-11-1941-42/102832",
    "https://www.zerozero.pt/edicao/ii-divisão-grupo-d-série-13-1942-43/133104",
    "https://www.zerozero.pt/edicao/ii-divisão-grupo-d-série-13-sub-série-3-1943-44/133174",
    "https://www.zerozero.pt/edicao/ii-divisão-grupo-d-série-16-1944-45/133238",
    "https://www.zerozero.pt/edicao/ii-divisão-grupo-d-série-16-1945-46/133436",
    "https://www.zerozero.pt/edicao/ii-divisão-grupo-d-série-16-1946-47/133754",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-d-1948-49/133777",
    "https://www.zerozero.pt/edicao/ii-divisao-zona-d-serie-8-1949-50/133823",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-d-1950-51/133838",
    "https://www.zerozero.pt/edicao/ii-divisão-grupo-sul-zona-d-1951-52/133858",
    "https://www.zerozero.pt/edicao/iii-divisao-grupo-sul-zona-d-serie-7-1952-53/133931",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-c-1953-1954/74671",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1954-1955/74678",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1955-1956/74686",
    "https://www.zerozero.pt/edicao/ii-divisao-zona-sul-1956-57/71218",
    "https://www.zerozero.pt/edicao/ii-divisao-zona-sul-1957-58/74987",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1958-59/74994",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1959-60/74998",
    "https://www.zerozero.pt/edicao/nacional-da-ii-divisão-zona-sul-1960-61/75029",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1961-62/75031",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1962-1963/56928",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1963-1964/57151",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1964-1965/57804",
    "https://www.zerozero.pt/edicao/iii-divisão-zona-d-série-8-1965-66/133932",
    "https://www.zerozero.pt/edicao/iii-divisao-zona-sul-serie-8-1966-67/133933",
    "https://www.zerozero.pt/edicao/iii-divisao-zona-d-serie-8-1967-68/133934",
    "https://www.zerozero.pt/edicao/iii-divisao-zona-d-1968-69/133935",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-69-70/75057",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1970-71/103",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisao-1971-72/104",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisao-1972-73/105",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1973-74/107",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1974-75/109",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1975-76/111",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1976-1977/75137",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1977-1978/75138",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1978-1979/75162",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1979-1980/15303",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1980-1981/75205",
    "https://www.zerozero.pt/edicao/ii-divisão-zona-sul-1981-1982/75226",
    "https://www.zerozero.pt/edicao/ii-divisao-zona-sul-1982-1983/70399",
    "https://www.zerozero.pt/edicao/ii-divisão-fase-final-1982-83/24184",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1983-84/89",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisao-1984-85/92",
    "https://www.zerozero.pt/edicao/ii-divisao-zona-sul-1985-86/21521",
    "https://www.zerozero.pt/edicao/ii-divisao-fase-final-1985-1986/21594",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1986-87/96",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1987-88/98",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1988-89/100",
    "https://www.zerozero.pt/edicao/ii-divisao-zona-sul-1989-1990/64919",
    "https://www.zerozero.pt/edicao/ii-divisao-ii-fase-1989-90/29612",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1990-91/120",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1991-92/122",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1992-93/124",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1993-94/123",
    "https://www.zerozero.pt/edicao/campeonato-nacional-da-i-divisão-1994-95/121",
    "https://www.zerozero.pt/edicao/liga-portuguesa-1995-96/106",
    "https://www.zerozero.pt/edicao/liga-portuguesa-1996-97/108",
    "https://www.zerozero.pt/edicao/liga-portuguesa-1997-98/110",
    "https://www.zerozero.pt/edicao/liga-portuguesa-1998-99/112",
    "https://www.zerozero.pt/edicao/liga-portuguesa-1999-00/114",
    "https://www.zerozero.pt/edicao/liga-portuguesa-2000-01/116",
    "https://www.zerozero.pt/edicao/liga-portuguesa-2001-02/119",
    "https://www.zerozero.pt/edicao/ii-liga-2002-2003/1081",
    "https://www.zerozero.pt/edicao/ii-divisão-b-zona-sul-2003-2004/6265",
    "https://www.zerozero.pt/edicao/iii-divisao-serie-f-2004-2005/546"
]

for url in urls_competicoes:
    titulo, subtitulo = extrair_titulo_subtitulo(url)
    df = extrair_classificacao(url)
    
    if df is not None:
        if subtitulo:
            display(Markdown(f"**{subtitulo}**"))
        display(Markdown(f"**{titulo}**"))
        display(Markdown(df.to_markdown(index=False)))
    else:
        print(f"Erro ao processar a classificação para {url}.")

import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, Markdown
import time
import random

def extrair_titulo_subtitulo(url_competicao):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url_competicao, headers=headers)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, 'html.parser')
        
        subtitle = soup.select_one("#zz-enthdr > div.zz-enthdr-top > div.zz-enthdr-data > div.text")
        title = soup.select_one("#zz-enthdr > div.zz-enthdr-top > div.zz-enthdr-data > h1 > span")
        
        title = title.text.strip() if title else '<span style="color:red;">Título não encontrado</span>'
        subtitle = subtitle.text.strip() if subtitle else None
        
        return title, subtitle
    except:
        return '<span style="color:red;">Erro ao carregar a página</span>', None

def extrair_classificacao(url_competicao):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        time.sleep(random.uniform(1, 3))
        
        response = requests.get(url_competicao, headers=headers)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, 'html.parser')
        
        table_element = soup.find('table', class_='zztable stats zz-datatable')
        
        if not table_element:
            edition_table_div = soup.find('div', id='edition_table')
            if edition_table_div:
                table_element = edition_table_div.find('table')
        
        if not table_element:
            for table in soup.find_all('table'):
                headers = table.find_all('th')
                if headers and len(headers) >= 10:
                    header_text = [h.text.strip() for h in headers]
                    if 'P' in header_text and 'J' in header_text and 'V' in header_text:
                        table_element = table
                        break
        
        if not table_element:
            raise ValueError("Tabela de classificação não encontrada.")
        
        rows = table_element.find_all('tr')
        data = []
        for row in rows[1:]:
            cols = row.find_all('td')
            if len(cols) >= 10:
                row_data = [col.text.strip() for col in cols[:10]]
                data.append(row_data)
        
        df = pd.DataFrame(data, columns=['Posição', ' ', 'Equipa', 'Pontos', 'V', 'E', 'D', 'GM', 'GS', 'DG'])
        
        return df
    
    except requests.RequestException as e:
        print(f"Erro na requisição HTTP para {url_competicao}: {e}")
    except ValueError as e:
        print(f"Erro ao processar os dados de {url_competicao}: {e}")
    except Exception as e:
        print(f"Erro inesperado ao extrair classificação de {url_competicao}: {e}")
    
    return None

# Lista de URLs das competições
urls_competicoes = [
    "https://www.zerozero.pt/edicao/af-algarve-1-divisao-2007-08/1697",
    "https://www.zerozero.pt/edicao/iii-divisao-serie-f-2008-2009/2379",
    # ... (other URLs)
    "https://www.zerozero.pt/edicao/liga-portugal-betclic-2023-24/175797"
]

# Extrair título e classificação para cada URL e exibir
for url in urls_competicoes:
    titulo, subtitulo = extrair_titulo_subtitulo(url)
    df = extrair_classificacao(url)
    
    if df is not None:
        if subtitulo:
            display(Markdown(f"**{subtitulo}**"))
        display(Markdown(f"**{titulo}**"))
        display(Markdown(df.to_markdown(index=False)))
    else:
        print(f"Erro ao processar a classificação para {url}.")

**II Liga 1934/35**

**II 1ªF Zona D Grupo 8 34/35**

|   Posição |     | Equipa              |   Pontos |   V |   E |   D |   GM |   GS |   DG |
|----------:|:----|:--------------------|---------:|----:|----:|----:|-----:|-----:|-----:|
|         1 |     | Olhanense           |       14 |   8 |   7 |   0 |    1 |   19 |   12 |
|         2 |     | Lusitano VRSA       |        8 |   8 |   3 |   2 |    3 |   24 |   16 |
|         3 |     | Luso Beja           |        7 |   8 |   3 |   1 |    4 |   12 |   14 |
|         4 |     | Lusitano Évora 1911 |        6 |   8 |   2 |   2 |    4 |   13 |   20 |
|         5 |     | Farense             |        5 |   8 |   1 |   3 |    4 |   12 |   18 |

**II Liga 1935/36**

**II Divisão Zona D 1ªF Grupo 2 35/36**

...

**III Divisão 08/09**

**III Divisão Série F 2008/2009**

|   Posição |     | Equipa          |   Pontos |   V |   E |   D |   GM |   GS |   DG |
|----------:|:----|:----------------|---------:|----:|----:|----:|-----:|-----:|-----:|
|         1 |     | Louletano       |       48 |  10 |   5 |   3 |    2 |   18 |   10 |
|         2 |     | At. Reguengos   |       43 |  10 |   5 |   3 |    2 |   21 |   16 |
|         3 |     | Farense         |       35 |  10 |   3 |   5 |    2 |   13 |   15 |
|         4 |     | CD Cova Piedade |       34 |  10 |   0 |   7 |    3 |    7 |   11 |
|         5 |     | Pescadores      |       32 |  10 |   3 |   3 |    4 |   14 |   17 |
|         6 |     | Juventude Évora |       30 |  10 |   2 |   3 |    5 |   12 |   16 |

**Liga Portugal Betclic 2023/24**

|   Posição |     | Equipa        |   Pontos |   V |   E |   D |   GM |   GS |   DG |
|----------:|:----|:--------------|---------:|----:|----:|----:|-----:|-----:|-----:|
|         1 |     | Sporting      |       90 |  34 |  29 |   3 |    2 |   96 |   29 |
|         2 |     | Benfica       |       80 |  34 |  25 |   5 |    4 |   77 |   28 |
|         3 |     | FC Porto      |       72 |  34 |  22 |   6 |    6 |   63 |   27 |
|         4 |     | SC Braga      |       68 |  34 |  21 |   5 |    8 |   71 |   50 |
|         5 |     | Vitória SC    |       63 |  34 |  19 |   6 |    9 |   52 |   38 |
|         6 |     | Moreirense    |       55 |  34 |  16 |   7 |   11 |   36 |   35 |
|         7 |     | FC Arouca     |       46 |  34 |  13 |   7 |   14 |   54 |   50 |
|         8 |     | FC Famalicão  |       42 |  34 |  10 |  12 |   12 |   37 |   41 |
|         9 |     | Casa Pia AC   |       38 |  34 |  10 |   8 |   16 |   38 |   50 |
|        10 |     | Farense       |       37 |  34 |  10 |   7 |   17 |   46 |   51 |
|        11 |     | Rio Ave       |       37 |  34 |   6 |  19 |    9 |   38 |   43 |
|        12 |     | Gil Vicente   |       36 |  34 |   9 |   9 |   16 |   42 |   52 |
|        13 |     | Estoril Praia |       33 |  34 |   9 |   6 |   19 |   49 |   58 |
|        14 |     | Est. Amadora  |       33 |  34 |   7 |  12 |   15 |   33 |   53 |
|        15 |     | Boavista      |       32 |  34 |   7 |  11 |   16 |   39 |   62 |
|        16 |     | Portimonense  |       32 |  34 |   8 |   8 |   18 |   39 |   72 |
|        17 |     | FC Vizela     |       26 |  34 |   5 |  11 |   18 |   36 |   66 |
|        18 |     | GD Chaves     |       23 |  34 |   5 |   8 |   21 |   31 |   72 |

# Mostra todos os resultados

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.auto import tqdm
from tabulate import tabulate
from IPython.display import display, Markdown
import re

# Criando dicionário de épocas
epocas = {}
for i in range(63, 156):
    epoca = i - 63
    ano_inicio = 1933 + epoca
    ano_fim = ano_inicio + 1
    epocas[i] = f"{ano_inicio}-{str(ano_fim)[-2:]}"

base_url = 'https://www.zerozero.pt/equipa/farense/10/jogos?grp=1&epoca_id='
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Função para corrigir caracteres especiais e limpar texto
def corrigir_caracteres_especiais(texto):
    # Substitui caracteres problemáticos por equivalentes
    texto_corrigido = texto.replace('ZS', 'Zona Sul').replace('ZD', 'Zona D').replace('Campees', 'Campeões').replace('Taa', 'Taça')
    # Remove espaços em branco extras e caracteres indesejados
    texto_corrigido = re.sub(r'\s+', ' ', texto_corrigido).strip()
    return texto_corrigido

# Função para exibir uma tabela formatada com os jogos
def exibir_tabela_jogos(competicao, epoca_nome, jogos):
    competicao = corrigir_caracteres_especiais(competicao)
    display(Markdown(f"---\n### Época {epoca_nome}"))
    display(Markdown(f"---\n### {competicao}"))
    df = pd.DataFrame(jogos)
    df = df.sort_values(by='Data', ascending=True)
    display(Markdown(tabulate(df, headers='keys', tablefmt='pipe', showindex=False)))

# Usando tqdm para mostrar a linha de progresso
for epoca_id, epoca_nome in tqdm(epocas.items(), desc="Processando épocas", unit="época"):
    url = f'{base_url}{epoca_id}'

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Decodificando conteúdo com tratamento de erros
        response.encoding = response.apparent_encoding  # Garantir a codificação correta
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todas as linhas relevantes na tabela
        linhas = soup.find_all('tr', class_='parent')

        competicoes = {}  # Reiniciar competições para cada época

        for linha in linhas:
            celulas = linha.find_all('td')
            if len(celulas) >= 8:
                competicao = celulas[7].text.strip()
                resultado = celulas[0].text.strip()
                if resultado == "h2h":
                    resultado = "-"
                local = celulas[3].text.strip()
                if local == '(F)':
                    local = 'Fora'
                elif local == '(C)':
                    local = 'Casa'
                
                jogo = {
                    'V-E-D': resultado,
                    'Data': celulas[1].text.strip(),
                    'Hora': celulas[2].text.strip(),
                    'Local': local,
                    'Equipa': celulas[5].text.strip(),
                    'Resultado': celulas[6].text.strip(),
                    'Jornada': celulas[8].text.strip()
                }

                if competicao in competicoes:
                    competicoes[competicao].append(jogo)
                else:
                    competicoes[competicao] = [jogo]

        # Exibir os jogos para cada competição, assim que os dados forem obtidos
        if competicoes:
            for competicao, jogos in competicoes.items():
                exibir_tabela_jogos(competicao, epoca_nome, jogos)
        else:
            display(Markdown(f"**Sem jogos encontrados para a época {epoca_nome}.**"))

    except requests.RequestException as e:
        display(Markdown(f"Erro ao acessar {url}: {e}"))

display(Markdown("### Processamento completo!"))


Processando épocas:   0%|          | 0/93 [00:00<?, ?época/s]

---
### Época 1933-34

---
### Campeonato do Algarve 1933/34

| V-E-D   | Data       | Hora   | Local   | Equipa       | Resultado   | Jornada   |
|:--------|:-----------|:-------|:--------|:-------------|:------------|:----------|
| V       | 1934-07-01 | 18:00  | Casa    | Portimonense | 4-1         | F         |

---
### Época 1934-35

...

---
### Época 2024-25

---
### Taça Portugal 24/25

| V-E-D   | Data       | Hora   | Local   | Equipa         | Resultado   | Jornada   |
|:--------|:-----------|:-------|:--------|:---------------|:------------|:----------|
| -       | 2024-10-20 | 14:00  | Fora    | AD Sanjoanense | -           | 3E        |

**Sem jogos encontrados para a época 2025-26.**

### Processamento completo!

# Mostra todas as equipas

In [3]:
import requests
from bs4 import BeautifulSoup
import os
from IPython.display import display, Markdown

# Criando dicionário de épocas
epocas = {}
for i in range(40, 155):
    epoca = i - 40
    ano_inicio = 1910 + epoca
    ano_fim = ano_inicio + 1
    epocas[i] = f"{ano_inicio}/{str(ano_fim)[-2:]}"

base_url = 'https://www.zerozero.pt/equipa/farense/10/jogos?grp=1&epoca_id='

def extrair_info_jogadores(epoca_id, epoca_nome):
    url = f"https://www.zerozero.pt/equipa/farense/10?epoca_id={epoca_id}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Decodificando conteúdo com tratamento de erros
    response.encoding = response.apparent_encoding  # Garantir a codificação correta
    soup = BeautifulSoup(response.text, 'html.parser')

    jogadores = []

    for posicao in soup.find_all("div", class_="innerbox"):
        # Verificar se a div com a classe "title" existe
        title_div = posicao.find("div", class_="title")
        if title_div is not None:
            posicao_nome = title_div.text.strip()
        else:
            posicao_nome = "Posição desconhecida"

        for jogador in posicao.find_all("div", class_="staff"):
            nome = jogador.find("div", class_="text").text.strip()

            # Verificar se o 'span' (idade) existe
            idade_element = jogador.find("span")
            idade = idade_element.text.strip() if idade_element else "Idade desconhecida"
            
            nacionalidade = jogador.find("img")["title"]

            jogadores.append({
                "nome": nome,
                "posicao": posicao_nome,
                "idade": idade,
                "nacionalidade": nacionalidade,
            })

    return jogadores

# Iterar sobre todas as épocas e mostrar as informações dos jogadores
for epoca_id, epoca_nome in epocas.items():
    display(Markdown("---"))
    display(Markdown(f"### Época {epoca_nome}"))
    info_jogadores = extrair_info_jogadores(epoca_id, epoca_nome)
    
    for jogador in info_jogadores:
        display(Markdown(f"{jogador['posicao']} |\n"
                         f"**{jogador['nome']}** \n"
                         f"| {jogador['idade']} |\n"
                         f"**Nacionalidade**: {jogador['nacionalidade']}"))

---

### Época 1910/11

Guarda Redes |
**João Rodrigues** 
| - |
**Nacionalidade**: Portugal

Defesa |
**José Nugas** 
| - |
**Nacionalidade**: Portugal

Médio |
**António Gago** 
| 13 anos |
**Nacionalidade**: Portugal

...

Presidente |
**João Rodrigues** 
| Idade desconhecida |
**Nacionalidade**: Portugal

Outros Dirigentes |
**Carlos Silva** 
| Diretor Desportivo |
**Nacionalidade**: Portugal

Outros Dirigentes |
**José Pedro Coelho** 
| Diretor de Comunicação |
**Nacionalidade**: Portugal

Outros Dirigentes |
**José Luís Domingos** 
| Diretor Geral de Futebol |
**Nacionalidade**: Portugal

Outros Dirigentes |
**Hugo Freire** 
| Gestor Operações |
**Nacionalidade**: Portugal

# Estatísticas de jogadores por época

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.auto import tqdm
from tabulate import tabulate
from IPython.display import display, Markdown
import re

# Criando dicionário de épocas
epocas = {}
for i in range(40, 155):
    epoca = i - 40
    ano_inicio = 1910 + epoca
    ano_fim = ano_inicio + 1
    epocas[i] = f"{ano_inicio}/{str(ano_fim)[-2:]}"

base_url = 'https://www.zerozero.pt/equipa/farense/10/jogadores?epoca_stats_id='
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Função para corrigir caracteres especiais e limpar texto
def corrigir_caracteres_especiais(texto):
    texto_corrigido = texto.replace('ZS', 'Zona Sul').replace('ZD', 'Zona D').replace('Campees', 'Campeões').replace('Taa', 'Taça')
    texto_corrigido = re.sub(r'\s+', ' ', texto_corrigido).strip()
    return texto_corrigido

# Função para exibir uma tabela formatada com as estatísticas dos jogadores
def exibir_tabela_estatisticas(epoca_nome, estatisticas):
    display(Markdown(f"---\n### Época {epoca_nome}"))
    df = pd.DataFrame(estatisticas)
    df = df.sort_values(by='J', ascending=False)
    display(Markdown(tabulate(df, headers='keys', tablefmt='pipe', showindex=False)))

# Usando tqdm para mostrar a linha de progresso
for epoca_id, epoca_nome in tqdm(epocas.items(), desc="Processando épocas", unit="época"):
    url = f'{base_url}{epoca_id}'

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar a tabela de estatísticas dos jogadores
        tabela = soup.find('table', class_='zztable stats zz-table')
        linhas = tabela.find_all('tr')[1:]  # Ignorar o cabeçalho

        estatisticas = []

        for linha in linhas:
            celulas = linha.find_all('td')
            if len(celulas) >= 13:
                jogador = {
                    'Nome': celulas[2].text.strip(),
                    'Posição': celulas[3].text.strip(),
                    'J': celulas[4].text.strip(),
                    'GM': celulas[5].text.strip(),
                    'MG': celulas[6].text.strip(),
                    'T': celulas[7].text.strip(),
                    'SU': celulas[8].text.strip(),
                    'M': celulas[9].text.strip(),
                    'A': celulas[10].text.strip(),
                    'AA': celulas[11].text.strip(),
                    'V': celulas[12].text.strip()
                }
                estatisticas.append(jogador)

        # Exibir estatísticas se encontradas
        if estatisticas:
            exibir_tabela_estatisticas(epoca_nome, estatisticas)
        else:
            display(Markdown(f"**Sem estatísticas de jogadores encontradas para a época {epoca_nome}.**"))

    except requests.RequestException as e:
        display(Markdown(f"Erro ao acessar {url}: {e}"))

display(Markdown("### Processamento completo!"))


Processando épocas:   0%|          | 0/115 [00:00<?, ?época/s]

---
### Época 1910/11

| Nome           | Posição      |   J |   GM |   MG | T   | SU   | M   | A   | AA   | V   |
|:---------------|:-------------|----:|-----:|-----:|:----|:-----|:----|:----|:-----|:----|
| João Rodrigues | Guarda Redes |   0 |    0 |    0 | -   | -    | -   | -   | -    | -   |
| José Nugas     | Defesa       |   0 |    0 |    0 | -   | -    | -   | -   | -    | -   |
| António Gago   | Médio        |   0 |    0 |    0 | -   | -    | -   | -   | -    | -   |

---
### Época 1911/12

| Nome           | Posição      |   J |   GM |   MG | T   | SU   | M   | A   | AA   | V   |
|:---------------|:-------------|----:|-----:|-----:|:----|:-----|:----|:----|:-----|:----|
| José Nugas     | Defesa       |   0 |    0 |    0 | -   | -    | -   | -   | -    | -   |
| António Gago   | Médio        |   0 |    0 |    0 | -   | -    | -   | -   | -    | -   |
| João Rodrigues | Guarda Redes |   0 |    0 |    0 | -   | -    | -   | -   | -    | -   |
| João Gralho    | Médio        |   0 |    0 |    0 | -   | -    | -   | -   | -    | -   |

...

---
### Época 2023/24

| Nome              | Posição      |   J |   GM |   MG | T   | SU   | M    | A   | AA   | V   |
|:------------------|:-------------|----:|-----:|-----:|:----|:-----|:-----|:----|:-----|:----|
| André Seruca      | Médio        |   9 |    0 | 0    | -   | 9    | 100  | -   | -    | -   |
| Marco Matias      | Avançado     |  38 |    3 | 0.08 | 33  | 5    | 2674 | 4   | -    | -   |
| Gonçalo Silva     | Defesa       |  37 |    1 | 0.03 | 37  | -    | 3238 | 3   | -    | 1   |
| Mattheus Oliveira | Médio        |  37 |    7 | 0.19 | 31  | 6    | 2485 | 6   | -    | 1   |
| Bruno Duarte      | Avançado     |  36 |   14 | 0.39 | 27  | 9    | 2356 | 6   | -    | -   |
| Fabrício Isidoro  | Médio        |  36 |    3 | 0.08 | 22  | 14   | 1882 | 8   | 1    | -   |
| Mohamed Belloumi  | Avançado     |  35 |    7 | 0.2  | 27  | 8    | 2164 | 3   | -    | -   |
| Cláudio Falcão    | Médio        |  35 |    4 | 0.11 | 34  | 1    | 2962 | 12  | -    | -   |
| Ricardo Velho     | Guarda Redes |  35 |    0 | 0    | 35  | -    | 3150 | 3   | -    | -   |
| Pastor            | Defesa       |  32 |    0 | 0    | 30  | 2    | 2662 | 9   | 1    | -   |
| Elves Baldé       | Avançado     |  32 |    1 | 0.03 | 10  | 22   | 1080 | -   | -    | -   |
| Rafael Barbosa    | Médio        |  31 |    1 | 0.03 | 10  | 21   | 1004 | 1   | -    | -   |
| Talocha           | Defesa       |  30 |    1 | 0.03 | 26  | 4    | 2333 | 3   | -    | -   |
| Vítor Gonçalves   | Médio        |  27 |    2 | 0.07 | 7   | 20   | 794  | 2   | 1    | -   |
| Zach Muscat       | Defesa       |  25 |    2 | 0.08 | 22  | 3    | 1920 | 5   | -    | 1   |
| Rui Costa         | Avançado     |  25 |    5 | 0.2  | 9   | 16   | 952  | 2   | -    | -   |
| Facundo Cáseres   | Médio        |  23 |    0 | 0    | 14  | 9    | 1252 | 6   | 1    | -   |
| Jhon Velásquez    | Médio        |   2 |    0 | 0    | -   | 2    | 23   | -   | -    | -   |
| Miguel Carvalho   | Guarda Redes |   2 |    0 | 0    | 2   | -    | 180  | -   | -    | -   |
| Luiz Felipe       | Guarda Redes |   2 |    0 | 0    | 2   | -    | 180  | 1   | -    | -   |
| Cristian Ponde    | Avançado     |  18 |    1 | 0.06 | 3   | 15   | 463  | 1   | -    | -   |
| Zé Luís           | Avançado     |  18 |    3 | 0.17 | 7   | 11   | 787  | 3   | -    | -   |
| Talys Oliveira    | Defesa       |  15 |    0 | 0    | 13  | 2    | 1164 | 1   | -    | -   |
| Igor Rossi        | Defesa       |  14 |    0 | 0    | 12  | 2    | 1103 | 4   | -    | -   |
| Artur Jorge       | Defesa       |  12 |    0 | 0    | 7   | 5    | 695  | 4   | -    | -   |
| Fran Delgado      | Defesa       |  12 |    0 | 0    | 9   | 3    | 761  | 1   | -    | -   |
| Maxuel Cássio     | Avançado     |   1 |    0 | 0    | -   | 1    | 33   | 1   | -    | -   |
| Diogo Paulo       | Médio        |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |
| Tiago Madeira     | Avançado     |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |
| Miguel Bandarra   | Defesa       |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |
| Hamidou Diallo    | Defesa       |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |
| Kauan Firmino     | Guarda Redes |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |
| André Candeias    | Avançado     |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |
| Adewale Sapara    | Avançado     |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |
| Arian Kastrati    | Avançado     |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |
| Rafael Teixeira   | Médio        |   0 |    0 | 0    | -   | -    | -    | -   | -    | -   |

---
### Época 2024/25

| Nome             | Posição      |   J |   GM |   MG | T   | SU   | M   | A   | AA   | V   |
|:-----------------|:-------------|----:|-----:|-----:|:----|:-----|:----|:----|:-----|:----|
| Ricardo Velho    | Guarda Redes |   8 |    0 | 0    | 8   | -    | 720 | -   | -    | -   |
| Ângelo Neto      | Médio        |   8 |    0 | 0    | 6   | 2    | 423 | 3   | -    | -   |
| Cláudio Falcão   | Médio        |   8 |    0 | 0    | 8   | -    | 720 | 3   | -    | -   |
| Álex Millán      | Avançado     |   6 |    0 | 0    | 3   | 3    | 303 | -   | -    | -   |
| Lucas Áfrico     | Defesa       |   6 |    0 | 0    | 6   | -    | 526 | 2   | -    | -   |
| Pastor           | Defesa       |   6 |    0 | 0    | 5   | 1    | 462 | -   | -    | -   |
| Darío Poveda     | Avançado     |   6 |    0 | 0    | 2   | 4    | 267 | -   | -    | -   |
| Álex Bermejo     | Avançado     |   6 |    1 | 0.17 | 3   | 3    | 283 | 1   | -    | -   |
| Elves Baldé      | Avançado     |   5 |    0 | 0    | 2   | 3    | 173 | -   | -    | -   |
| Miguel Menino    | Médio        |   5 |    0 | 0    | 2   | 3    | 225 | -   | -    | -   |
| Filipe Soares    | Médio        |   5 |    0 | 0    | 3   | 2    | 204 | -   | -    | -   |
| Rafael Barbosa   | Médio        |   5 |    0 | 0    | 3   | 2    | 304 | 2   | -    | -   |
| Tomané           | Avançado     |   5 |    1 | 0.2  | 4   | 1    | 362 | 1   | -    | -   |
| Marco Moreno     | Defesa       |   5 |    0 | 0    | 5   | -    | 403 | 1   | -    | -   |
| Artur Jorge      | Defesa       |   5 |    0 | 0    | 5   | -    | 450 | 4   | -    | -   |
| Rivaldo Morais   | Defesa       |   4 |    0 | 0    | 4   | -    | 360 | -   | -    | -   |
| Marco Matias     | Avançado     |   4 |    0 | 0    | 2   | 2    | 156 | -   | -    | -   |
| Paulo Victor     | Defesa       |   4 |    0 | 0    | -   | 4    | 66  | -   | -    | -   |
| Derick Poloni    | Defesa       |   4 |    0 | 0    | 4   | -    | 305 | -   | -    | -   |
| Mohamed Belloumi | Avançado     |   3 |    0 | 0    | 3   | -    | 270 | -   | -    | -   |
| Raul Silva       | Defesa       |   3 |    1 | 0.33 | 3   | -    | 257 | 2   | -    | -   |
| Talys Oliveira   | Defesa       |   3 |    0 | 0    | 3   | -    | 253 | -   | -    | -   |
| Mehdi Merghem    | Avançado     |   3 |    0 | 0    | 3   | -    | 221 | -   | -    | -   |
| Geovanny Almeida | Médio        |   3 |    0 | 0    | 1   | 2    | 106 | -   | -    | -   |
| Jhon Velásquez   | Médio        |   2 |    0 | 0    | -   | 2    | 13  | -   | -    | -   |
| André Candeias   | Avançado     |   2 |    0 | 0    | -   | 2    | 34  | -   | -    | -   |
| Cuba             | Avançado     |   2 |    0 | 0    | -   | 2    | 16  | -   | -    | -   |
| Jaime Pinto      | Avançado     |   1 |    0 | 0    | -   | 1    | 38  | -   | -    | -   |
| Kaique           | Guarda Redes |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| Luiz Felipe      | Guarda Redes |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| Miguel Carvalho  | Guarda Redes |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| Rafael Teixeira  | Médio        |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| André Seruca     | Médio        |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| Tiago Madeira    | Avançado     |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| Talocha          | Defesa       |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| Kauan Firmino    | Guarda Redes |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| Lucas Cañizares  | Guarda Redes |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |
| Frederico Silva  | Médio        |   0 |    0 | 0    | -   | -    | -   | -   | -    | -   |

### Processamento completo!